In [23]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
import pytz
import pickle
from tqdm.auto import tqdm
import os


/home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
input_base_path = '../obtaining_news_collections/data/'
input_file_names = [x for x in os.listdir(input_base_path) if 'dedup.csv' in x]
print(input_file_names)

['newyork_article_texts_and_info_dedup.csv', 'nytimes_foxnews_article_texts_and_info_dedup.csv', 'florida_article_texts_and_info_dedup.csv', 'illinois_article_texts_and_info_dedup.csv', 'texas_article_texts_and_info_dedup.csv', 'california_article_texts_and_info_dedup.csv', 'ohio_article_texts_and_info_dedup.csv']


In [ ]:
#for text, using title + article text combined. 
#output will be indexed using url, and each state and (nytimes foxnews) will be done separately to create 7 files
#each file is a dictionary, from url -> (timestamp, [(score, keyword/phrase), ...])

In [27]:
def get_dated_keywords(ts_text_tuples, 
                       min_length=1, 
                       max_length=3):
    """
    Given ts_text_tuples, a list of (timestamp, article_text) tuples,
    extract & return a list of timestamped keywords + scores using rake-nltk. 
    """
    dated_keywords = []
    r = Rake(min_length=min_length, max_length=max_length)
    for tup in tqdm(ts_text_tuples):
        r.extract_keywords_from_text(tup[1])
        kw = r.get_ranked_phrases_with_scores()
        dated_keywords.append((tup[0], kw))
    
    return dated_keywords

In [29]:
for f in input_file_names:#, position = 0, leave=False):
    media_group = f.split('_article_')[0]
    print(media_group)
    df = pd.read_csv(input_base_path + f)
    urls = list(df['url'])
    dates = list(df['publish_date'])
    titles_and_texts = list(zip(df['title'], df['text']))
    titles_and_texts = list(map(lambda x:x[0] + '\n\n\n' + x[1], titles_and_texts))
    dated_keywords = get_dated_keywords(list(zip(dates, titles_and_texts)))
    assert len(dated_keywords)==len(urls)
    out = dict(zip(urls, dated_keywords))
    pickle.dump(out,
                open(media_group + '_url_to_dated_keywords.pkl',
                          'wb'))

newyork


100%|██████████| 2910/2910 [00:08<00:00, 357.26it/s]


nytimes_foxnews


100%|██████████| 944/944 [00:01<00:00, 538.24it/s]


florida


...

100%|██████████| 1816/1816 [00:04<00:00, 431.22it/s]


california


100%|██████████| 7571/7571 [00:30<00:00, 244.33it/s]


ohio


100%|██████████| 667/667 [00:02<00:00, 322.44it/s]
